In [ ]:
# Temporary imports cell to ensure kernel has required modules loaded
import sys
print('Python', sys.version)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.inspection import permutation_importance
import joblib
import warnings
warnings.filterwarnings('ignore')
print('Imports done')


# Decision Tree and Random Forest training notebook

This notebook builds, evaluates, visualizes, tunes, and saves Decision Tree and Random Forest models for the
`cybersecurity_intrusion_data.csv` dataset. It follows the project's preprocessing conventions (numeric scaling + one-hot
encoding for categoricals) and saves models to the repository `models/` folder.


In [ ]:
# Section 1: Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.inspection import permutation_importance
import joblib
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
sns.set(style='whitegrid')


In [ ]:
# Section 2: Load dataset
# Notebook-friendly path (workspace relative). Adjust if you open the notebook from a different working directory.
DATA_PATH = Path('../../Sheets/cybersecurity_intrusion_data.csv').resolve()
print('Loading data from:', DATA_PATH)
df = pd.read_csv(DATA_PATH)
print('Data shape:', df.shape)
df.head()


In [ ]:
# Section 3: Exploratory data check & preprocessing notes
print('\n--- Basic info ---')
print(df.info())

print('\n--- Description (numeric) ---')
print(df.describe().T)

print('\n--- Class distribution ---')
print(df['attack_detected'].value_counts())
print(df['attack_detected'].value_counts(normalize=True))

# Basic missing value check
print('\n--- Missing values per column ---')
print(df.isnull().sum())

# Define feature groups
numeric_features = [
    'network_packet_size',
    'login_attempts',
    'session_duration',
    'ip_reputation_score',
    'failed_logins',
    'unusual_time_access'
]
categorical_features = ['protocol_type', 'encryption_used', 'browser_type']

# Keep only available columns (defensive)
available = set(df.columns)
numeric_features = [c for c in numeric_features if c in available]
categorical_features = [c for c in categorical_features if c in available]

print('\nNumeric features:', numeric_features)
print('Categorical features:', categorical_features)


In [ ]:
# Section 4: Preprocessing pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Build preprocessor similar to scripts/train_models.py
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ],
    remainder='drop'
)

# Fit the preprocessor to allow extraction of feature names later
preprocessor.fit(df[numeric_features + categorical_features])

# Derive feature names after preprocessing (for feature importance plots)
feature_names = []
if numeric_features:
    feature_names.extend(numeric_features)
if categorical_features:
    ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
    cat_names = list(ohe.get_feature_names_out(categorical_features))
    feature_names.extend(cat_names)

print('\nPreprocessed feature count:', len(feature_names))


In [ ]:
# Section 5: Train / test split
TARGET = 'attack_detected'
X = df[numeric_features + categorical_features]
y = df[TARGET].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print('Train shape:', X_train.shape)
print('Test shape:', X_test.shape)
print('\nTrain class balance:')
print(y_train.value_counts(normalize=True))


In [ ]:
# Section 6: Decision Tree — train and evaluate
# Build pipeline combining the preprocessor and the tree classifier
from sklearn.tree import DecisionTreeClassifier

dt_pipeline = Pipeline(steps=[('preproc', preprocessor), ('clf', DecisionTreeClassifier(random_state=RANDOM_STATE))])

# Fit
dt_pipeline.fit(X_train, y_train)

# Predict & evaluate
y_pred = dt_pipeline.predict(X_test)
print('\nDecision Tree — Test set results')
print(classification_report(y_test, y_pred, digits=4))
print('Confusion matrix:')
print(confusion_matrix(y_test, y_pred))

# Cross-validated ROC AUC (pipeline will apply preprocessing inside CV)
cv_scores = cross_val_score(dt_pipeline, X, y, cv=5, scoring='roc_auc', n_jobs=-1)
print('\nCross-validated ROC AUC (5-fold):')
print(cv_scores)
print('Mean ROC AUC:', np.mean(cv_scores))


In [ ]:
# Section 7: Decision Tree — visualize tree (shallow depth for readability)
import matplotlib.pyplot as plt
clf = dt_pipeline.named_steps['clf']

# Transform a small sample to get the correct shape
X_test_trans = dt_pipeline.named_steps['preproc'].transform(X_test)

plt.figure(figsize=(18,8))
plot_tree(clf, max_depth=3, filled=True)
plt.title('Decision Tree (top levels)')
plt.show()


In [ ]:
# Section 8: Decision Tree — feature importance & pruning
# Feature importances (note: these correspond to the preprocessed feature ordering)
import pandas as pd
import numpy as np

imp = clf.feature_importances_
if len(imp) == len(feature_names):
    fi = pd.Series(imp, index=feature_names).sort_values(ascending=False)
else:
    fi = pd.Series(imp).sort_values(ascending=False)

print('\nTop features (Decision Tree):')
print(fi.head(15))

# Cost-complexity pruning path
X_train_trans = dt_pipeline.named_steps['preproc'].transform(X_train)
path = clf.cost_complexity_pruning_path(X_train_trans, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Train trees using the different alphas
clfs = []
for ccp_alpha in ccp_alphas[:10]:  # limit for speed
    clf_tmp = DecisionTreeClassifier(random_state=RANDOM_STATE, ccp_alpha=ccp_alpha)
    clf_tmp.fit(X_train_trans, y_train)
    clfs.append((ccp_alpha, clf_tmp))

print('\nTrained {} pruned trees (sample of alphas)'.format(len(clfs)))


In [ ]:
# Section 9: Random Forest — train and evaluate
rf_pipeline = Pipeline(steps=[('preproc', preprocessor), ('clf', RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, oob_score=True, n_jobs=-1))])
rf_pipeline.fit(X_train, y_train)

# Predict & evaluate
y_pred_rf = rf_pipeline.predict(X_test)
print('\nRandom Forest — Test set results')
print(classification_report(y_test, y_pred_rf, digits=4))
print('Confusion matrix:')
print(confusion_matrix(y_test, y_pred_rf))

# OOB score
rf_clf = rf_pipeline.named_steps['clf']
if hasattr(rf_clf, 'oob_score_'):
    print('\nRandom Forest OOB score:', rf_clf.oob_score_)

# Feature importances
imp_rf = rf_clf.feature_importances_
if len(imp_rf) == len(feature_names):
    fi_rf = pd.Series(imp_rf, index=feature_names).sort_values(ascending=False)
else:
    fi_rf = pd.Series(imp_rf).sort_values(ascending=False)

print('\nTop features (Random Forest):')
print(fi_rf.head(15))

# Permutation importance (optional, can be slower)
perm = permutation_importance(rf_pipeline, X_test, y_test, n_repeats=10, random_state=RANDOM_STATE, n_jobs=-1)
perm_idx = perm.importances_mean.argsort()[::-1]
print('\nTop permutation importances:')
for i in perm_idx[:10]:
    name = feature_names[i] if i < len(feature_names) else f'X{i}'
    print(f"{name}: {perm.importances_mean[i]:.4f} (+/- {perm.importances_std[i]:.4f})")


In [ ]:
# Section 10: Hyperparameter tuning (GridSearchCV example for Random Forest)
param_grid = {
    'clf__n_estimators': [50, 100],
    'clf__max_depth': [None, 10, 20],
    'clf__max_features': ['sqrt', 'log2']
}

grid = GridSearchCV(rf_pipeline, param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)
print('\nBest params (GridSearch):', grid.best_params_)
print('Best CV ROC AUC:', grid.best_score_)

best_rf = grid.best_estimator_
# Evaluate on test set
y_pred_grid = best_rf.predict(X_test)
print('\nGrid Seached RF — Test set')
print(classification_report(y_test, y_pred_grid, digits=4))
print('Confusion matrix:')
print(confusion_matrix(y_test, y_pred_grid))


In [ ]:
# Section 11: Save models and notebook export notes
MODEL_DIR = Path('../../models').resolve()
MODEL_DIR.mkdir(parents=True, exist_ok=True)

joblib.dump(dt_pipeline, MODEL_DIR / 'decision_tree_model.joblib')
joblib.dump(rf_pipeline, MODEL_DIR / 'random_forest_model.joblib')
# Save the best grid-searched model if available
try:
    joblib.dump(best_rf, MODEL_DIR / 'random_forest_best_grid.joblib')
except NameError:
    pass

print('Models saved to:', MODEL_DIR)

# Notebook export note (run in integrated terminal if you want a clean exported copy):
# jupyter nbconvert --to notebook --output exported_train_tree_models.ipynb train_tree_models.ipynb
